In [2]:
import pandas as pd
import logging as log

%run Metric.ipynb
%run DataSource.ipynb
%run Account.ipynb
%run Risk.ipynb
%run Market.ipynb
%run PolicyCons.ipynb
%run Policy_Loader.ipynb
%run Rocket_Operation.ipynb
%run Two_MA_Operation.ipynb
%run Evaluation_Metric.ipynb


class Operation:
    
    # symbol=SZ#002335
    def __init__(self, account, symbol, context, policy_name):
        self.context = context
        
        self.account = account
        self.symbol = symbol
        self.short = context['short']
        self.mid = context['mid']
        self.policy_name = policy_name
        
        self.operation_detail = pd.DataFrame(columns=operation_columns)
        
        self.metric = Metric(self.symbol, self.context)
        self.risk = Risk(self.account,self.symbol,self.context,self.policy_name)
        self.market = Market(self.context)
        self.policy_loader = Policy_Loader(self.policy_name)
        self.debug = self.policy_loader.enabled_log_debug()
        
        self.pCons = PolicyCons()
        self.evaluation_Metric = Evaluation_Metric(self.account)

        strategy = self.policy_loader.get_exec_strategy()
        if(strategy==self.pCons.break_strategy):
            self.exec_chief = Rocket_Operation(account, context, self.metric, self.policy_name)
        else:
            self.exec_chief =  Two_MA_Operation(account, context, self.metric, self.policy_loader)
        

    def sell_how_many(self,cur_date):
        return self.exec_chief.sell_how_many(cur_date)
    
    def open_opsition(self,cur_date):
        price = self.metric.get_cur_price(cur_date)
        if(price is None):
            return 0
        
        account_cash = self.account.get_all_cash()
        
        buy_shares = self.risk.buy_how_many(cur_date,price)
        
        if(buy_shares>0):
            reason = self.get_buy_reason(cur_date)
            self.account.buy_stock(self.symbol,cur_date,price,buy_shares,reason)
        
        return buy_shares
        
        
    def sell_stock(self,cur_date, sell_type='sell_point'):
        sell_price = self.get_sell_price(cur_date)
        sell_shares = self.sell_how_many(cur_date)
        
        profit_rate = round((sell_price-self.account.get_open_position_price(self.symbol))*100/self.account.get_open_position_price(self.symbol),2)
        
        open_price = self.account.get_open_position_price(self.symbol)
        open_date = self.account.get_open_position_date(self.symbol)
        
        if(sell_shares>0):
            reason = self.get_sell_reason(cur_date)
            self.account.sell_stock(self.symbol,cur_date,sell_price,sell_shares,reason)
        
        space_rate = self.account.get_space_per_of_account(cur_date)
        asset_value = self.account.get_today_stock_asset(cur_date)+self.account.get_all_cash()
        
        market_status = self.market.get_market_status(cur_date)
        
        drawdown = self.evaluation_Metric.get_float_profit_drawdown()
        operation_log = [self.symbol,open_date,cur_date,open_price,sell_price,
                        sell_shares,space_rate,
                        profit_rate,asset_value,market_status,drawdown]
        if(self.debug==True):
            print('operation_log\n' + str(operation_log))
        self.operation_detail.loc[self.operation_detail.shape[0]+1] = operation_log
        
    
    def get_operations(self):
        return self.operation_detail
    
    def is_ma_cross_buy_point(self,cur_date):
        return self.metric.is_up_cross(cur_date)
    
    def is_ma_cross_sell_point(self,cur_date):
        return self.metric.is_down_cross(cur_date)
    
        
    def is_ma_cross_buy_point_delay_confirm(self,cur_date):
        latest_n=3
        up_crossed = self.latest_n_period_up_crossed(cur_date, latest_n)
        keep_up = self.k_on_ma(cur_date, self.short)

        if(up_crossed == True and keep_up == True):
            if(debug):
                print(cur_date+' up cross and keep 3 k lines up on MA('+ str(short)+')')
            return True
        else:
            return False
        
    def is_ma_cross_sell_point_delay_confirm(self, cur_date):
        down_cross = self.is_down_cross(cur_date)

        latest_n = 3
        latest_n_break_ma = self.latest_n_k_break_ma(cur_date, latest_n)

        deep_percent=3
        deep_break_ma = self.deep_k_break_ma(cur_date,deep_percent)

        if(down_cross == True):
            if(debug):
                print('down_cross')
            return True

        if(latest_n_break_ma == True):
            if(debug):
                print('latest_'+str(latest_n)+'_break_ma_'+str(mid))
            return True

        if(deep_break_ma == True):
            if(debug):
                print('deep_'+str(deep_percent)+'%_break_ma_'+str(mid))
            return True

        return False
    
    
#   固定增长幅度止盈
    def is_stop_profit_by_fixed_percent(self,cur_date,threshold=50):
        if(self.policy_loader.enabled_stop_profit()==False):
            return False
        
        price = self.metric.get_cur_price(cur_date)
        last_buy_price = self.account.get_open_position_price(self.symbol)
        
        profit_rate = 100*(price-last_buy_price)/last_buy_price
        if(profit_rate>threshold):
            return True
        else:
            return False
    
    
    def which_sell_type(self,cur_date):
        return self.exec_chief.which_sell_type(cur_date)
        
    def is_buy_point(self,cur_date):
        return self.exec_chief.is_buy_point(cur_date)
        
    def get_buy_reason(self,cur_date):
        return self.exec_chief.get_buy_reason(cur_date)
        
    def is_sell_point(self, cur_date):
        return self.exec_chief.is_sell_point(cur_date)
        
    def get_sell_reason(self,cur_date):
        return self.exec_chief.get_sell_reason(cur_date)
        
    def get_sell_price(self,cur_date):
        return self.exec_chief.get_sell_price(cur_date)

    def is_over_stop_loss(self,cur_date):
        if(self.policy_loader.enabled_stop_loss()==False):
            return False
        
        stop_loss_price = self.risk.get_over_stop_loss_price(cur_date)
        price = self.metric.get_cur_price(cur_date)
        
        if(price<stop_loss_price):
            return True
        else:
            return False


    def send_go_signal(self,outburst, pre_amplitude, pre_ma_per, amplitude_threshold=60):
        if((outburst==True) and (pre_ma_per>-10)):
            return 'Buy'

        return 'Hold'
    
    def get_vol_features(self,cur_date):
        vol = self.get_vol(cur_date)
        
        vol_features = []
        for long in [10,20,40,60]:
            vol_features.append(int(vol/self.ma_vol(cur_date,long)))
            for short in [2,4,8,16]:
                temp_feature = int(self.ma_vol(cur_date,short)/self.ma_vol(cur_date,long))
                vol_features.append(temp_feature)

        return vol_features
    
    
